In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
import heapq
from math import pow
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

## Extremely good LM

### Build a flat distribution

In [ ]:
def flat_distribution(min_entropy=9, prompt="."):
    dist = [(-1, prompt)]
    while dist[0][0] < -pow(2,-min_entropy):
        # get the highest probability string
        (minus_max_prob, prompt) = heapq.heappop(dist)
        # evaluate the model on the highest probability string
        inputs = tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"]
        next_token_logits = top_k_top_p_filtering(model(**inputs).logits[:, -1, :], top_k=int(pow(2,min_entropy)))
        probs = nn.functional.softmax(next_token_logits, dim=-1)
        # enter the new strings into the distribution map
        nz = torch.nonzero(probs)
        for i in nz:
            new_input_ids = torch.cat([input_ids, torch.tensor([[i[1]]])], dim=-1)
            new_prompt = tokenizer.decode(new_input_ids.tolist()[0])
            heapq.heappush(dist,(minus_max_prob*probs[0,i[1]], new_prompt))
    return [(-p,s) for (p,s) in dist]

## Encryption

In [ ]:
#keygen
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_GCM)
nonce = cipher.nonce
#encrypt
ciphertext, tag = cipher.encrypt_and_digest(b"fish")
encode_this = nonce+ciphertext+tag

## Sample until hash function matches

In [ ]:
s = 99
def hash(inp, seed=0):
    return inp % 256

In [ ]:
def encode_byte(current_string, b):
    dist_and_strings = flat_distribution(prompt=current_string)
    dist = torch.tensor([p for (p,_) in dist_and_strings])
    print("distribution done")

    while True:
        sample = torch.multinomial(dist,1)
        #print(sample)
        #print(hash(sample))
        if hash(sample) == b:
            print(sample)
            return dist_and_strings[sample][1]

In [ ]:
s = ""
for b in encode_this:
    print("The text currently is :", s)
    k = input("Finish the current sentence.")
    s += k.split(".",2)[0].split("?",2)[0].split("!",2)[0]
    s = encode_byte(s,b)

## Decryption

In [ ]:
#decrypt
cipher = AES.new(key, AES.MODE_GCM,nonce)
cipher.decrypt(ciphertext)